# Stitching Tutorial

The Workflow of the Stitching Pipeline can be seen in the following. Note that the image comes from the [OpenCV Documentation](https://docs.opencv.org/3.4/d1/d46/group__stitching.html).

With the following block, we allow displaying resulting images within the notebook:

In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from pathlib import Path

from stitching.image_handler import ImageHandler
from stitching.feature_detector import FeatureDetector
from stitching.feature_matcher import FeatureMatcher
from stitching.subsetter import Subsetter
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector
from stitching.warper import Warper
from stitching.timelapser import Timelapser

In [2]:
def get_image_paths(img_set):
    return sorted([str(path.relative_to('.')) for path in Path('../../data/sequential_training').rglob(img_set)])

In [3]:
def get_displacement(img_paths):
    img_handler = ImageHandler()
    img_handler.set_img_names(img_paths)

    medium_imgs = list(img_handler.resize_to_medium_resolution())
    final_imgs = list(img_handler.resize_to_final_resolution())

    finder = FeatureDetector()
    features = [finder.detect_features(img) for img in medium_imgs]

    matcher = FeatureMatcher()
    matches = matcher.match_features(features)

    subsetter = Subsetter()

    indices = subsetter.get_indices_to_keep(features, matches)

    medium_imgs = subsetter.subset_list(medium_imgs, indices)
    final_imgs = subsetter.subset_list(final_imgs, indices)
    features = subsetter.subset_list(features, indices)
    matches = subsetter.subset_matches(matches, indices)

    camera_estimator = CameraEstimator()
    camera_adjuster = CameraAdjuster()
    wave_corrector = WaveCorrector()

    cameras = camera_estimator.estimate(features, matches)
    cameras = camera_adjuster.adjust(features, matches, cameras)
    cameras = wave_corrector.correct(cameras)

    warper = Warper()
    warper.set_scale(cameras)

    final_sizes = img_handler.get_final_img_sizes()
    camera_aspect = img_handler.get_medium_to_final_ratio()    # since cameras were obtained on medium imgs

    warped_final_imgs = list(warper.warp_images(final_imgs, cameras, camera_aspect))
    final_corners, final_sizes = warper.warp_rois(final_sizes, cameras, camera_aspect)

    def get_first_nonzero_pixel(ndarr):
        return np.argwhere(ndarr > 0)[0][:2]

    def get_center(corner, size):
        return (round(corner[0] + size[1] / 2), round(corner[1] + size[0] / 2))

    timelapser = Timelapser('as_is')
    timelapser.initialize(final_corners, final_sizes)
    centers = []
    for img, corner, size in zip(warped_final_imgs, final_corners, final_sizes):
        timelapser.process_frame(img, corner)
        frame = timelapser.get_frame()
        centers.append(get_center(get_first_nonzero_pixel(frame), size))
    
    centers = np.array(centers)
    return centers, np.diff(centers, axis=0)

In [4]:
sequential_imgs = get_image_paths('*.jpg')

In [20]:
get_displacement(sequential_imgs[:5])

(array([[112, 112],
        [224, 112],
        [224, 224]]), array([[112,   0],
        [  0, 112]]))

In [5]:
disps = []
for p1, p2 in zip(sequential_imgs, sequential_imgs[1:]):
    _, disp = get_displacement([p1, p2])
    disps.append(disp)

disps = np.array(disps)
disps

StitchingError: No match exceeds the given confidence threshold.